# <span style="color: #FF6347; font-family: 'Arial', sans-serif;">**Face recognition project**</span>


In [27]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

path = 'pictures'
images = []
playersNames = []
players_list = os.listdir(path)

for plyer_class in players_list:
    curImg = cv2.imread(os.path.join(path, plyer_class))
    images.append(curImg)
    plyer_name_without_extension = plyer_class.split('.')[0]
    playersNames.append(plyer_name_without_extension)

def findEncodings(images):
    encodeList = []
    for img in images:
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = findEncodings(images)
print('Encoding Complete')

def markAttendance(name):
    with open('attendance.csv', 'r+') as f:
        # Read the current data from the file
        myDataList = f.readlines()
        # Get the names from the file
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])

        # If the name is not already in the list, add it
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%Y-%m-%d %H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    success, img = cap.read()
    if not success:
        print("Error: Failed to capture image from webcam.")
        break

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if faceDis[matchIndex] < 0.60:  # Check if the face distance is below threshold
            name = playersNames[matchIndex]
        else:
            name = 'Unknown'

        y1, x2, y2, x1 = faceLoc
        y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

        # Mark attendance
        markAttendance(name)

    cv2.imshow('Webcam', img)

    # Wait for 'q' key to close webcam
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Properly release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


Encoding Complete
